In [1]:
from theano import *
import theano.tensor as T
from theano import function
from theano import pp


In [2]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x,y], z)

In [3]:
f(2,3)

array(5.0)

In [4]:
numpy.allclose(f(16.3, 12.1), 28.4)

True

In [5]:
print(pp(z))

(x + y)


In [6]:
a = theano.tensor.vector()
out = a + a**10
f = theano.function([a], out)
print(f([0,1,2]))

[    0.     2.  1026.]


In [7]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[Elemwise{exp,no_inplace}(<TensorType(float64, vector)>)]
Looping 1000 times took 38.526862 seconds
Result is [ 1.23178032  1.61879341  1.52278065 ...,  2.20771815  2.29967753
  1.62323285]
Used the cpu


In [11]:
from theano import function, config, shared, sandbox
import theano.sandbox.cuda.basic_ops
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), 'float32'))
f = function([], sandbox.cuda.basic_ops.gpu_from_host(T.exp(x)))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
print("Numpy result is %s" % (numpy.asarray(r),))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

1 #define _CUDA_NDARRAY_C
2 
3 #include <Python.h>
4 #include <structmember.h>
5 
6 #include <numpy/arrayobject.h>
7 #include <iostream>
8 
9 #include "cuda_ndarray.cuh"
10 
11 //If true, when there is a gpu malloc or free error, we print the size of allocated memory on the device.
12 #define COMPUTE_GPU_MEM_USED 0
13 
14 //If true, we fill with NAN allocated device memory.
15 #define ALLOC_MEMSET 0
16 
17 //If true, we print out when we free a device pointer, uninitialize a
18 //CudaNdarray, or allocate a device pointer
19 #define PRINT_FREE_MALLOC 0
20 
21 //If true, we do error checking at the start of functions, to make sure there
22 //is not a pre-existing error when the function is called.
23 //You probably need to set the environment variable
24 //CUDA_LAUNCH_BLOCKING=1, and/or modify the CNDA_THREAD_SYNC
25 //preprocessor macro in cuda_ndarray.cuh
26 //if you want this to work.
27 #define PRECHECK_ERROR 0
28 
29 //If true, we release the GIL around blocking GPU calls, to allow 


['nvcc', '-shared', '-O3', '-m64', '-Xcompiler', '-DCUDA_NDARRAY_CUH=m11b90075e2397c684f9dc0f7276eab8f,-D NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC', '-Xlinker', '-rpath,/home/silver/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-3.5.1-64/cuda_ndarray', '-I/home/silver/anaconda3/lib/python3.5/site-packages/theano/sandbox/cuda', '-I/home/silver/anaconda3/lib/python3.5/site-packages/numpy/core/include', '-I/home/silver/anaconda3/include/python3.5m', '-o', '/home/silver/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-3.5.1-64/cuda_ndarray/cuda_ndarray.so', 'mod.cu', '-L/home/silver/anaconda3/lib', '-lpython3.5m', '-lcublas', '-lcudart']


OSError: ('The following error happened while compiling the node', GpuFromHost(Elemwise{exp,no_inplace}.0), '\n', 'You forced the use of gpu device gpu, but CUDA initialization failed with error:\ncuda unavilable')